In [3]:
# Proyek Training Model Deteksi Usia & Jenis Kelamin
# Skrip ini digunakan untuk melatih tiga arsitektur model yang berbeda 
# (ResNet50, MobileNetV2, EfficientNet) pada dataset usia dan jenis kelamin.

# ### Tahap 1: Impor Library & Pengaturan Awal ###
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import os
from tqdm import tqdm
import numpy as np

# Cek apakah GPU tersedia dan set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# ### Tahap 2: Persiapan Data (Dataset & DataLoader) ###
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

class GenderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        for label, gender in enumerate(['female', 'male']):
            gender_path = os.path.join(self.root_dir, gender)
            if not os.path.isdir(gender_path):
                continue
            for img_name in os.listdir(gender_path):
                self.image_paths.append(os.path.join(gender_path, img_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Warning: Could not load image {img_path}. Error: {e}. Skipping.")
            return self.__getitem__((idx + 1) % len(self))
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.float32)

class AgeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        for age_folder in os.listdir(self.root_dir):
            folder_path = os.path.join(self.root_dir, age_folder)
            if os.path.isdir(folder_path):
                try:
                    parts = age_folder.split('-')
                    avg_age = (int(parts[0]) + int(parts[1])) / 2.0 if len(parts) == 2 else float(age_folder)
                    for img_name in os.listdir(folder_path):
                        self.image_paths.append(os.path.join(folder_path, img_name))
                        self.labels.append(avg_age)
                except ValueError:
                    continue

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Warning: Could not load image {img_path}. Error: {e}. Skipping.")
            return self.__getitem__((idx + 1) % len(self))
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.float32)

# --- PATH DATASET ---
GENDER_DATA_PATH = './dataset/gender/Training/'
AGE_DATA_PATH = './dataset/age/Training/'
# --------------------------------------

gender_dataset = GenderDataset(root_dir=GENDER_DATA_PATH, transform=data_transforms['train'])
age_dataset = AgeDataset(root_dir=AGE_DATA_PATH, transform=data_transforms['train'])

gender_loader = DataLoader(gender_dataset, batch_size=32, shuffle=True, num_workers=0)
age_loader = DataLoader(age_dataset, batch_size=32, shuffle=True, num_workers=0)

print(f"Jumlah gambar gender ditemukan: {len(gender_dataset)}")
print(f"Jumlah gambar usia ditemukan: {len(age_dataset)}")

FileNotFoundError: [WinError 3] The system cannot find the path specified: './dataset/age/Training/'

In [ ]:
# ### Tahap 3: Definisi Arsitektur Model ###

# Model 1: ResNet50
class ResNet50AgeGenderModel(nn.Module):
    def __init__(self): 
        super(ResNet50AgeGenderModel, self).__init__()
        # Gunakan pretrained=True untuk transfer learning yang lebih baik
        self.base_model = models.resnet50(pretrained=True)
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()
        self.gender_head = nn.Sequential(nn.Linear(in_features, 512), nn.ReLU(), nn.Dropout(0.5), nn.Linear(512, 1))
        self.age_head = nn.Sequential(nn.Linear(in_features, 512), nn.ReLU(), nn.Dropout(0.5), nn.Linear(512, 1))
    def forward(self, x):
        features = self.base_model(x)
        return self.gender_head(features), self.age_head(features)

# Model 2: MobileNetV2
class MobileNetV2AgeGenderModel(nn.Module):
    def __init__(self):
        super(MobileNetV2AgeGenderModel, self).__init__()
        self.base_model = models.mobilenet_v2(pretrained=True)
        in_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Identity()
        self.gender_head = nn.Sequential(nn.Linear(in_features, 256), nn.ReLU(), nn.Dropout(0.5), nn.Linear(256, 1))
        self.age_head = nn.Sequential(nn.Linear(in_features, 256), nn.ReLU(), nn.Dropout(0.5), nn.Linear(256, 1))
    def forward(self, x):
        features = self.base_model(x)
        return self.gender_head(features), self.age_head(features)

# Model 3: EfficientNet-B0
class EfficientNetAgeGenderModel(nn.Module):
    def __init__(self):
        super(EfficientNetAgeGenderModel, self).__init__()
        self.base_model = models.efficientnet_b0(pretrained=True)
        in_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Identity()
        self.gender_head = nn.Sequential(nn.Linear(in_features, 256), nn.ReLU(), nn.Dropout(0.5), nn.Linear(256, 1))
        self.age_head = nn.Sequential(nn.Linear(in_features, 256), nn.ReLU(), nn.Dropout(0.5), nn.Linear(256, 1))
    def forward(self, x):
        features = self.base_model(x)
        return self.gender_head(features), self.age_head(features)


In [ ]:
# ### Tahap 4: Fungsi Training & Validasi ###
def train_model(model, model_name, gender_loader, age_loader, num_epochs=10):
    model = model.to(device)
    
    # Loss functions dan optimizer
    criterion_gender = nn.BCEWithLogitsLoss()
    criterion_age = nn.L1Loss() # MAE
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    print(f'--- Memulai Training untuk {model_name} ---')

    for epoch in range(num_epochs):
        model.train()
        running_gender_loss = 0.0
        running_age_loss = 0.0
        gender_corrects = 0
        
        age_iter = iter(age_loader)
        progress_bar = tqdm(gender_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for gender_inputs, gender_labels in progress_bar:
            # Training Gender
            gender_inputs, gender_labels = gender_inputs.to(device), gender_labels.to(device).unsqueeze(1)
            optimizer.zero_grad()
            gender_outputs, _ = model(gender_inputs)
            loss_gender = criterion_gender(gender_outputs, gender_labels)
            
            # Training Usia
            try:
                age_inputs, age_labels = next(age_iter)
            except StopIteration:
                age_iter = iter(age_loader)
                age_inputs, age_labels = next(age_iter)
            age_inputs, age_labels = age_inputs.to(device), age_labels.to(device).unsqueeze(1)
            _, age_outputs = model(age_inputs)
            loss_age = criterion_age(age_outputs, age_labels)

            # Gabungkan loss dan backpropagate
            total_loss = loss_gender + loss_age
            total_loss.backward()
            optimizer.step()
            
            # Kalkulasi statistik
            running_gender_loss += loss_gender.item() * gender_inputs.size(0)
            running_age_loss += loss_age.item() * age_inputs.size(0)
            preds = torch.sigmoid(gender_outputs) > 0.5
            gender_corrects += torch.sum(preds == gender_labels.data)
            
            progress_bar.set_postfix(gender_loss=f"{loss_gender.item():.4f}", age_mae=f"{loss_age.item():.4f}")
        
        epoch_gender_loss = running_gender_loss / len(gender_loader.dataset)
        epoch_age_mae = running_age_loss / len(age_loader.dataset) # Menggunakan age_loader untuk perhitungan rata-rata yang lebih akurat
        epoch_gender_acc = gender_corrects.double() / len(gender_loader.dataset)
        
        print(f"Epoch {epoch+1}/{num_epochs} -> Train Results | Gender Loss: {epoch_gender_loss:.4f}, Gender Acc: {epoch_gender_acc:.4f}, Age MAE: {epoch_age_mae:.4f}")

    print(f'--- Training untuk {model_name} Selesai! ---')
    return model

In [ ]:
# ### Tahap 5: Eksekusi Training ###
# Pilih model yang ingin Anda latih dengan menghapus tanda komentar (`#`).
def main():
    # --- PILIH MODEL DAN NAMA FILE OUTPUT DI SINI ---

    # Opsi 1: Latih ResNet50
    # model_to_train = ResNet50AgeGenderModel()
    # output_filename = 'resnet50_age_gender.pth'

    # Opsi 2: Latih MobileNetV2
    # model_to_train = MobileNetV2AgeGenderModel()
    # output_filename = 'mobilenetv2_age_gender.pth'

    # Opsi 3: Latih EfficientNet
    model_to_train = EfficientNetAgeGenderModel()
    output_filename = 'efficientnet_age_gender.pth'

    # ---------------------------------------------

    if 'model_to_train' in locals():
        # Latih model yang dipilih
        trained_model = train_model(
            model=model_to_train, 
            model_name=output_filename.split('_')[0].upper(),
            gender_loader=gender_loader, 
            age_loader=age_loader, 
            num_epochs=10 # Anda bisa menambah jumlah epoch untuk hasil lebih baik
        )

        # Simpan model yang sudah dilatih
        torch.save(trained_model.state_dict(), output_filename)
        print(f"Model berhasil disimpan sebagai {output_filename}")
    else:
        print("Pilih salah satu model untuk dilatih dengan menghapus tanda komentar di dalam fungsi main().")


if __name__ == '__main__':
    main()